# Tensorflow Eager mode

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.datasets import make_moons
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import SparseCategoricalCrossentropy

assert tf.__version__ == '2.0.0-alpha0'

## Eager Enabled (Default)

In [ ]:
tf.executing_eagerly()

In [ ]:
x = [[2., 2.],
     [1., 0.]]

m = tf.matmul(x, x)

m

In [ ]:
m.numpy()

## Generate fake data

In [ ]:
X, y = make_moons(n_samples=1000,
                  noise=0.1,
                  shuffle=True,
                  random_state=0,
              )

In [ ]:
sns.scatterplot(x=X[:,0], y=X[:,1], hue=y);

## Model (Sequential)

In [ ]:
model = Sequential([
    Dense(8, input_shape=(2,), activation='tanh'),
    Dense(2, activation='tanh'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(lr=0.1),
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Model is a Function

In [ ]:
model(X[:10])

## Model (Functional)

In [ ]:
ins = Input(shape=(2, ))
x1 = Dense(16, activation='tanh')(ins)
x2 = Dense(8, activation='tanh')(x1)
x3 = Dense(2, activation='tanh')(x2)
outs = Dense(1, activation='sigmoid')(x3)

model1 = Model(inputs=ins, outputs=outs)
model2 = Model(inputs=ins, outputs=x3)


model1.compile(optimizer=Adam(lr=0.005),
               loss='binary_crossentropy',
               metrics=['accuracy'])

In [ ]:
model1.fit(X, y, epochs=30);

In [ ]:
model1(X[:10])

In [ ]:
X_inner = model2(X)

In [ ]:
sns.scatterplot(x=X_inner[:,0], y=X_inner[:,1], hue=y);

## Model (Class + manual loss)

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense_1 = Dense(16, activation='tanh')
        self.dense_2 = Dense(8, activation='tanh')
        self.dense_3 = Dense(2)
    def call(self, inputs):
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        return self.dense_3(x)

model3 = MyModel()

In [ ]:
model3(X[:10])

## Optimizer and Loss

In [ ]:
optimizer = Adam(lr=0.01)
loss = SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
logits = model3(X)

In [ ]:
loss(y, logits).numpy()

## Gradients

In [ ]:
def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        logits = model(inputs)
        loss_value = loss(targets, logits)
    return tape.gradient(loss_value, model.variables)

grad(model3, X, y)

## Dataset

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.shuffle(len(X)*2).repeat().batch(32)

## Training Loop

In [ ]:
loss_history = []
accuracy_history = []
acc = tf.keras.metrics.SparseCategoricalAccuracy()

for (i, (data, labels)) in enumerate(dataset.take(1000)):
    with tf.GradientTape() as tape:
        logits = model3(data)
        loss_value = loss(labels, logits)

    loss_history.append(loss_value.numpy())
    
    grads = tape.gradient(loss_value, model3.trainable_variables)
    
    optimizer.apply_gradients(zip(grads, model3.trainable_variables))
        
    acc.update_state(labels, logits)
    accuracy = acc.result().numpy()
    accuracy_history.append(accuracy)
    
    if i % 100 == 0:
        print("Iteration: {}, Loss: {:0.3}, Acc: {:0.3}".format(i, loss_value.numpy(), accuracy))

In [ ]:
plt.plot(loss_history)
plt.plot(accuracy_history)

## Flow Control

In [ ]:
def fizzbuzz(max_num):
    for i in tf.range(max_num):
        if tf.equal(i % 3, 0):
            print('Fizz')
        elif tf.equal(i % 5, 0):
            print('Buzz')
        else:
            print(i.numpy())

fizzbuzz(15)

## Tensorflow Function

In [ ]:
@tf.function
def square_if_positive(x):
    if x > 0:
        x = x * x
    else:
        x = 0
    return x

In [ ]:
print('square_if_positive(2) = {}'.format(square_if_positive(tf.constant(2))))
print('square_if_positive(-2) = {}'.format(square_if_positive(tf.constant(-2))))

## Disable Eager Mode

In [ ]:
tf.compat.v1.disable_eager_execution()

In [ ]:
tf.executing_eagerly()

In [ ]:
x = [[2., 2.],
     [1., 0.]]

m = tf.matmul(x, x)

m

In [ ]:
model(X)

In [ ]:
def fizzbuzz(max_num):
    for i in tf.range(max_num):
        if tf.equal(i % 3, 0):
            print('Fizz')
        elif tf.equal(i % 5, 0):
            print('Buzz')
        else:
            print(i)

fizzbuzz(15)

In [ ]:
@tf.function
def fizzbuzz(max_num):
    for i in tf.range(max_num):
        if tf.equal(i % 3, 0):
            print('Fizz')
        elif tf.equal(i % 5, 0):
            print('Buzz')
        else:
            print(i)

fizzbuzz(15)

In [ ]:
@tf.function
def square_if_positive(x):
    if x > 0:
        x = x * x
    else:
        x = 0
    return x

In [ ]:
print('square_if_positive(2) = {}'.format(square_if_positive(tf.constant(2))))
print('square_if_positive(-2) = {}'.format(square_if_positive(tf.constant(-2))))

In [ ]:
print(tf.autograph.to_code(fizzbuzz.python_function, experimental_optional_features=None))

## Exercise

As you've seen, now that we have disabled eager execution our previous models started to throw errors. Go ahead and create a new model `model4` and train it.

- create a sequential model, any architecture you'd like
- compile it
- fit it on `X` and `y` for 10 epochs
- run `model4(X)`
    - Does the model behave like a function?
    - Do you get the same results as you got above with `model(X)`?
- run `model4.predict(X)`:
    - what results do you get in this case?